In [ ]:
%matplotlib inline

In [ ]:
#%%writefile finding_biggest_lung.py

from 3DRegionGrowing.segmentation_functions import resample
#from read_files import normalization
import scipy.ndimage.interpolation as inter
import numpy as np
import os
import matplotlib.pyplot as plt
import scipy.io as spio
import string
import pydicom
from imutils import rotate
import time
import preprocess

def counting_mask_size(mask):

    #turn_boolean = np.array(mask, dtype=np.bool)
    turn_boolean = np.where(mask>0.5, True, False)
    #np.where(a < 5, a, 10*a)
    number = np.sum(turn_boolean)
    return number

def main():

    slices_path = "G:/CTimages/"
    indices_path ="G:/Plans/CT/Lungs/ok/Separated Lungs"
    indices_folder= os.listdir(indices_path)

    numero_min_voxels = float('Inf')
    numero_max_voxels = 0
    i=0
    for each_mask in indices_folder[4:5]:
        print(i)
        opened = spio.loadmat(indices_path+'/'+each_mask, squeeze_me=True)["indexes"]
        patient_id = each_mask.split(" ", each_mask.count(each_mask))[0] 
        print(patient_id)
        slices = preprocess.arrange_slices(slices_path,patient_id)
       
        normalized_volume=preprocess.normalization(slices)
        normalized_array = np.array(normalized_volume)
        mask = preprocess.creating_mask(opened,normalized_array)
        mask_resampled, spacing = preprocess.resample(mask, slices, [5,5,5])
        slide = mask_resampled.shape[1]*2//3 -1
        plt.imshow(mask_resampled[:,slide,:])
        this_counter = counting_mask_size(mask_resampled)
        turn_1 = np.where(mask_resampled>0.5, 1, 0)
        number = np.sum(turn_1)
        print("im counter: "+str(this_counter))
        print("im number: "+str(number))
        if this_counter>numero_max_voxels:
            numero_max_voxels=this_counter
        if this_counter<numero_min_voxels:
            numero_min_voxels=this_counter
        print('mask size: '+str(this_counter))
        print("max: "+str(numero_max_voxels))
        print("min: " + str(numero_min_voxels))
        i=i+1
        del slices
        #del mask_resampled
        del normalized_array
        del normalized_volume
        
    print("max global: "+str(numero_max_voxels))
    print("min global: " + str(numero_min_voxels))

    
if __name__=="__main__":
    main()